<a href="https://colab.research.google.com/github/Cinnamy/DocumentsClassification/blob/main/5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Подключаем модули

In [ ]:
import sklearn
import numpy as np
import nltk
import re
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models
from gensim.utils import simple_preprocess

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


#Загружаем и предобрабатываем данные

In [ ]:
stopwords_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
well_separated = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=['comp.graphics', 'rec.autos', 'sci.med', 'talk.politics.mideast'])
well_separated_full = []
well_separated_target = [[], [], []]
well_separated_nouns = []
well_separated_nouns_and_adjs = []
for i in range(300):
  text = well_separated.data[i]
  text = re.sub(r'[^(\w\s\-)]|[\d]|[()]|-{2,}', '', text)
  text = re.sub(r'\s{2,}|[\n]', ' ', text)
  text = [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(text) if word not in stopwords_en]
  if text != []:
    well_separated_full.append(text)
    well_separated_target[0].append(well_separated.target[i])
  nouns = [word[0] for word in nltk.pos_tag(text) if word[1] == "NN"]
  if nouns != []:
    well_separated_nouns.append(nouns)
    well_separated_target[1].append(well_separated.target[i])
  nouns_and_adjs = [word[0] for word in nltk.pos_tag(text) if word[1] == "NN" or word[1] == "JJ"]
  if nouns_and_adjs != []:
    well_separated_nouns_and_adjs.append(nouns_and_adjs)
    well_separated_target[2].append(well_separated.target[i])

#5 задание

##Bag of Words

In [ ]:
dictionary_full = corpora.Dictionary(well_separated_full)
dictionary_nouns = corpora.Dictionary(well_separated_nouns)
dictionary_nouns_and_adjs = corpora.Dictionary(well_separated_nouns_and_adjs)

bow_full = [dictionary_full.doc2bow(doc) for doc in well_separated_full]
bow_nouns = [dictionary_nouns.doc2bow(doc) for doc in well_separated_nouns]
bow_nouns_and_adjs = [dictionary_nouns_and_adjs.doc2bow(doc) for doc in well_separated_nouns_and_adjs]

##LDA

###Делаем векторизацию

In [ ]:
lda_model_full = models.LdaModel(bow_full, num_topics=4, id2word=dictionary_full, passes=15)
lda_model_nouns = models.LdaModel(bow_nouns, num_topics=4, id2word=dictionary_nouns, passes=15)
lda_model_nouns_and_adjs = models.LdaModel(bow_nouns_and_adjs, num_topics=4, id2word=dictionary_nouns_and_adjs, passes=15)

In [ ]:
X_full_LDA = []
for doc_bow in bow_full:
  document_topics = lda_model_full.get_document_topics(doc_bow, minimum_probability=0.0)
  document_topic_vector = [topic_prob for _, topic_prob in document_topics]
  X_full_LDA.append(document_topic_vector)

X_nouns_LDA = []
for doc_bow in bow_nouns:
  document_topics = lda_model_nouns.get_document_topics(doc_bow, minimum_probability=0.0)
  document_topic_vector = [topic_prob for _, topic_prob in document_topics]
  X_nouns_LDA.append(document_topic_vector)

X_nouns_and_adjs_LDA = []
for doc_bow in bow_nouns_and_adjs:
 document_topics = lda_model_nouns_and_adjs.get_document_topics(doc_bow, minimum_probability=0.0)
 document_topic_vector = [topic_prob for _, topic_prob in document_topics]
 X_nouns_and_adjs_LDA.append(document_topic_vector)

###Выводим метрики

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full_LDA, X_nouns_LDA, X_nouns_and_adjs_LDA]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=4, n_init=1).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X, kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X, kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(well_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans")

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.009418,0.750980,745.676923,0.443138
Nouns,0.054086,0.838996,1614.425892,0.238090
Nouns and adjs,0.096974,0.810963,1370.151647,0.276075


In [ ]:
aggl_euc_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=4, linkage=link).fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean")

In [ ]:
aggl_cos_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=4, linkage=link, metric='cosine').fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine")

##LSI

###Делаем векторизацию

In [ ]:
lsi_model_full = models.LsiModel(bow_full, id2word=dictionary_full, num_topics=4)
lsi_model_nouns = models.LsiModel(bow_nouns, id2word=dictionary_nouns, num_topics=4)
lsi_model_nouns_and_adjs = models.LsiModel(bow_nouns_and_adjs, id2word=dictionary_nouns_and_adjs, num_topics=4)

In [ ]:
X_full_LSI = []
for doc_bow in bow_full:
  document_topics = lda_model_full.get_document_topics(doc_bow, minimum_probability=0.0)
  document_topic_vector = [topic_prob for _, topic_prob in document_topics]
  X_full_LSI.append(document_topic_vector)

X_nouns_LSI = []
for doc_bow in bow_nouns:
  document_topics = lda_model_nouns.get_document_topics(doc_bow, minimum_probability=0.0)
  document_topic_vector = [topic_prob for _, topic_prob in document_topics]
  X_nouns_LSI.append(document_topic_vector)

X_nouns_and_adjs_LSI = []
for doc_bow in bow_nouns_and_adjs:
 document_topics = lda_model_nouns_and_adjs.get_document_topics(doc_bow, minimum_probability=0.0)
 document_topic_vector = [topic_prob for _, topic_prob in document_topics]
 X_nouns_and_adjs_LSI.append(document_topic_vector)

###Выводим метрики

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full_LSI, X_nouns_LSI, X_nouns_and_adjs_LSI]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=4, n_init=1).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X, kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X, kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(well_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans")

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.009500,0.748056,734.852419,0.451009
Nouns,0.054086,0.838998,1614.614142,0.238088
Nouns and adjs,0.096974,0.810926,1370.266178,0.276074


In [ ]:
aggl_euc_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=4, linkage=link).fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean")

In [ ]:
aggl_cos_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=4, linkage=link, metric='cosine').fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine")